In [1]:
folder= r"/Users/fabrizio/Documents/Datasett AI"
import regex as re
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as matplt
import json as js
from matplotlib.colors import LinearSegmentedColormap
from sklearn.manifold import TSNE
from os import walk
import collections
import string
from re import sub
from sklearn.decomposition import PCA
from itertools import groupby, chain
from matplotlib import cm
import nltk
import scipy as sp
from os import walk
import pickle
import tarfile
from collections import Counter
import datetime
import seaborn as sns
from dateutil import parser
from numpy import median
from scipy.stats import zscore
def import_data_excell (filenames,Type=str,Header=0, col_index=0):
    dataSheet=[];
    fileExcel= pd.ExcelFile(filenames);
    for sheet in fileExcel.sheet_names:
        if col_index==0:
            dataSheet.append(fileExcel.parse(sheet ,dtype=Type,header=Header))
        else:
            dataSheet.append(fileExcel.parse(sheet ,dtype=Type,header=Header,index_col=col_index))
    del sheet
    return dataSheet
def importDataset(filename,head):
    dataRaw=pd.read_csv(filename, ";",header=head, encoding= 'ISO-8859-1')
    return dataRaw 

not_merged=import_data_excell(folder+r"/Municipalities that have not been merged in 5 years.xlsx",Type=str)[0]
# not_merged[['komnr','komname']]=not_merged["region"].str.split(" ", n = 1, expand = True)  
not_merged=not_merged['Kommunenr']
kode=import_data_excell(folder+r"/Koder Kommuner 1994-.xlsx")[0]
kode["GRUPPEKODE"]=kode["GRUPPEKODE"].str.replace("K.", '')
# kode["KODE"]=[int(w) if str.isdigit(w) else w for w in kode["KODE"] ]
dict_kode=dict(zip(kode["KODE"],kode["GRUPPEKODE"]))
dict_kode_int=dict(zip(kode["KODE"].astype(float),kode["GRUPPEKODE"]))
name=kode["GRUPPETEKST"].str.replace(" ", "",regex=True)
name_alfabetical=name.str.replace("(", ",",regex=True)
name_alfabetical=name_alfabetical.str.replace(")", "",regex=True)

dict_kom_name_to_kode=dict(zip(name_alfabetical,kode["GRUPPEKODE"]))
print(name)
years=[ '2016',
'2017', '2018', '2019', '2020']#'2009', '2010', '2011', '2012', '2013', '2014', '2015',
years_int=[int(w) for w in years]

def import_data_excell_venteliste (filenames,Type=None,Header=0):
    years=['2018','2019','2020']
    fileExcel= pd.ExcelFile(filenames);
    y=0
    for sheet in fileExcel.sheet_names[1:-1]:
        temp=fileExcel.parse(sheet ,dtype=Type,header=Header)
        temp[("år",'')]=years[y]
        temp[("komnr",'')]=temp[('Unnamed: 0_level_0')]#.str.replace("K-", '',regex=False).astype(int)
        if y==0:
            dataSheet=temp
        else:
            dataSheet=pd.concat([dataSheet,temp],axis=0)
  
        y=y+1;
    del sheet
    return dataSheet.reset_index()
kode.to_csv("Komune list and code")
years=[ '2010', '2011', '2012', '2013', '2014', '2015','2016',
'2017', '2018', '2019', '2020','2021']

0             Halden
1             Halden
2               Moss
3               Moss
4               Moss
           ...      
732    Deltekommuner
733    Deltekommuner
734    Deltekommuner
735    Deltekommuner
736    Deltekommuner
Name: GRUPPETEKST, Length: 737, dtype: object


/opt/anaconda3/envs/nrk/lib/python3.7/site-packages/ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version.


In [3]:
årsverk=import_data_excell (r"/Users/fabrizio/Desktop/pdf_summary_NRK/Andel årsverk kvinner og menn per kommune.xlsx",Type=str,Header=0)[0]
årsverk=årsverk.query("kjonn=='A' and komfor in @kode.GRUPPEKODE").drop(["kommune","kjonn","andel","tekst"],axis=1)#komnr in @not_merged
årsverk=årsverk.pivot(index="komfor", columns="Tid", values="verk")
årsverk=årsverk.apply(pd.to_numeric, errors='coerce')
årsverk.columns=years
årsverk.index.name='komnr'
årsverk.to_csv("årsvekt.csv")

In [4]:
Users=import_data_excell(folder+r"/SSB/Merged serie/12003 Brukere av hjemmetjenesten.xlsx",Header=0)[0]
Users[['komnr','komname']]=Users["region"].str.split(" ", n = 1, expand = True) 
users=Users.query(" statistikkvariabel=='Brukere (antall)' and komnr in @kode.GRUPPEKODE")#komnr in @not_merged &'Brukere (antall)'per 1000 innbyggere
users[years[-6:-1]]=users[years[-6:-1]].apply(pd.to_numeric, errors='coerce')
users_helsetjenste=users.groupby(by=['komnr'], axis=0, level=None, as_index=True, sort=False).sum()
# no_users_kom=np.where(users_helsetjenste.sum(axis=1)==0)
# to_drop=users_helsetjenste.index[no_users_kom].values
# users_helsetjenste=users_helsetjenste.query("komnr not in @to_drop")
Users2021=import_data_excell(folder+r"/SSB/Merged serie/12003 Brukere av hjemmetjenesten - 2021.xlsx",Header=0)[0]
Users2021[['komnr','komname']]=Users2021["Region"].str.split(" ", n = 1, expand = True) 
users2021=Users2021#.query("komnr in @not_merged")
users2021["2021 - Brukere (antall)"]=users2021["2021 - Brukere (antall)"].apply(pd.to_numeric, errors='coerce')
users_helsetjenste2021=users2021.groupby(by=['komnr'], axis=0, level=None, as_index=True, sort=False).sum()
users_helsetjenste["2021"]=users_helsetjenste2021["2021 - Brukere (antall)"]#2021 - Brukere (antall)
users_helsetjenste.to_csv("users.csv")

Årsverk_user_ratio=årsverk.iloc[:,-6:].divide(users_helsetjenste)
Årsverk_user_ratio.to_csv("Årsvekt_per_user.csv")


/opt/anaconda3/envs/nrk/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [5]:
ages=["67-79 år","80 år og over"] 
users_67_plus=Users.query(" statistikkvariabel=='Brukere (antall)' and alder in @ages and komnr in @kode.GRUPPEKODE")#komnr in @not_merged &'Brukere (antall)'per 1000 innbyggere
users_67_plus[years[-6:-1]]=users_67_plus[years[-6:-1]].apply(pd.to_numeric, errors='coerce')
users_helsetjenste_67_plus=users_67_plus.groupby(by=['komnr'], axis=0, level=None, as_index=True, sort=False).sum()
# no_users_kom=np.where(users_helsetjenste_67_plus.sum(axis=1)==0)
# to_drop=users_helsetjenste_67_plus.index[no_users_kom].values
# users_helsetjenste_67_plus=users_helsetjenste_67_plus.query("komnr not in @to_drop")

users2021_67_plus=Users2021
users2021_67_plus["2021 - Brukere (antall)"]=users2021.query("Alder in @ages and komnr in @kode.GRUPPEKODE")["2021 - Brukere (antall)"].apply(pd.to_numeric, errors='coerce')


users_helsetjenste2021_67_plus=users2021_67_plus.groupby(by=['komnr'], axis=0, level=None, as_index=True, sort=False).sum()
users_helsetjenste_67_plus["2021"]=users_helsetjenste2021_67_plus["2021 - Brukere (antall)"]#2021 - Brukere (antall)
users_helsetjenste_67_plus.to_csv("users_over_67.csv")

/opt/anaconda3/envs/nrk/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [6]:
plass_list=importDataset(folder+r"/SSB/Merged serie/13232 Helse- og omsorgsinstitusjoner - plasser - 2020-kommunestruktur, etter statistikkvariabel, region og år.csv",1)
plass_list[['komnr','komname']]=plass_list["region"].str.replace("K-", '',regex=False).str.split(" ", n = 1, expand = True) 

plass_list= plass_list.replace({"komnr":dict_kode},regex=True)
plass_list_public=plass_list.query("statistikkvariabel=='Sykehjem - disponible plasser (antall)' and komnr in @kode.GRUPPEKODE")
plass_list_public.drop(['statistikkvariabel', 'region','komname'],axis=1,inplace=True)
plass_list_public.set_index(['komnr'], drop=True, append=False, inplace=True, verify_integrity=True)
plass_list_public=plass_list_public.apply(pd.to_numeric, errors='coerce')
plass_list_public.to_csv("plass_list.csv")
# plass_list_tot=plass_list.query("statistikkvariabel=='Sykehjem - kommunale plasser (antall)' or statistikkvariabel=='Sykehjem - private disponible plasser (antall)'")
# plass_list_tot=plass_list_tot.apply(pd.to_numeric, errors='coerce').replace({"komnr":dict_kode_int},regex=True)
# plass_list_tot=plass_list_tot.groupby(by=['komnr'], axis=0, level=None, as_index=True, sort=False).sum()
# plass_list_tot.drop(['statistikkvariabel', 'region','komname'],axis=1,inplace=True)
# plass_list_tot=plass_list_tot.query("komnr in @not_merged ")
# print(plass_list_tot["2020"])
befolkning_69=importDataset(folder+r"/SSB/Merged serie/07459 Befolkning, etter statistikkvariabel, region, kjønn, alder og år.csv",1)
befolkning_69[['komnr','komname']]=befolkning_69["region"].str.replace("K-", '',regex=False).str.split(" ", n = 1, expand = True) 
befolkning_69=befolkning_69.query("alder=='67 år eller eldre' and komnr in @kode.GRUPPEKODE ")
befolkning_69.drop(['statistikkvariabel', 'region','komname',"kjønn","alder"],axis=1,inplace=True)
befolkning_69=befolkning_69.apply(pd.to_numeric, errors='coerce').replace({"komnr":dict_kode_int},regex=True).groupby(by=['komnr'], axis=0, level=None, as_index=False, sort=False).sum()#.query("komnr in @not_merged ")
befolkning_69=befolkning_69.replace({"komnr":dict_kode},regex=True)
befolkning_69.set_index(['komnr'], drop=True, append=False, inplace=True, verify_integrity=True)
befolkning_69.to_csv("befolkning_69.csv")

/opt/anaconda3/envs/nrk/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  """Entry point for launching an IPython kernel.
/opt/anaconda3/envs/nrk/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/envs/nrk/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  app.launch_new_instance()


In [4]:
ed_levels=import_data_excell(folder+r"/SSB/Merged serie/Workers education F234+F254.xlsx",Header=0)[0]
ed_levels[['komnr','komname']]=ed_levels["Region"].str.split(" ", n = 1, expand = True) 
ed_levels=ed_levels.query("komnr in @kode.GRUPPEKODE and Sektor=='Total'")#
ed_level_set={"Sykepleier u/ videreutdanning":"high",
"Sykepleier med videreutdanning eksl. jordmor/helsesykepleier":"high", 
"Helsefagarbeider, hjelpepleier, omsorgsarbeider":"low",
"Pleiemedhjelper/pleieassistent (mm) m/annen helseutdanning":"low",
"Personell uten helse- og sosialfaglig utdanning":"low"}
ed_levels=ed_levels.replace(ed_level_set,regex=True).query("Yrkesgruppe == 'high' or Yrkesgruppe == 'low'").groupby(by=["komnr","År","Yrkesgruppe"], axis=0, level=None, as_index=False, sort=False).sum(min_count=1)
ed_levels=ed_levels[['komnr', 'År','Av. årsverk ekskl. lange fravær','Yrkesgruppe']]
ed_H=ed_levels.query("Yrkesgruppe=='high'").pivot_table(index=['komnr'],columns='År',values='Av. årsverk ekskl. lange fravær')
ed_L=ed_levels.query("Yrkesgruppe=='low'").pivot_table(index=['komnr'],columns='År',values='Av. årsverk ekskl. lange fravær')
ed_H.dropna(axis=0,how="all",thresh=3,inplace=True)
ed_L.dropna(axis=0,how="all",thresh=3,inplace=True)
ed_h_percentage=ed_H.divide(ed_H+ed_L)
ed_h_percentage.dropna(axis=0,how="all",inplace=True)
ed_ratio=ed_H.divide(ed_L)
ed_ratio=ed_ratio.replace(np.inf, np.nan)
ed_ratio.dropna(axis=0,how="all",inplace=True)
ed_ratio.to_csv("education_level.csv")

In [5]:
ed_h_percentage.to_csv("education_percentage.csv")

In [6]:
ed_H.to_csv("education_High.csv")
ed_ratio.to_csv("education_Low.csv")

In [9]:
lonnsutgifter=import_data_excell(folder+"/SSB/Merged serie/12362 Lønnsutgifter - Hjemmeboende F254 - 2020-2021.xlsx",Type=str,Header=3)[0]
lonnsutgifter[['komnr','komname']]=lonnsutgifter.iloc[:,0].str.replace("K-", '',regex=False).str.split(" ", n = 1, expand = True) 
lonnsutgifter=lonnsutgifter.replace({"komnr":dict_kode},regex=True)
lonn_all=lonnsutgifter.query("komnr in @kode.GRUPPEKODE")
lonn_all.set_index(['komnr'], drop=True, append=False, inplace=True, verify_integrity=True)
lonn_all=lonn_all[["2020","2021"]].apply(pd.to_numeric, errors='coerce')
lonnsutgifter=lonnsutgifter.query("komnr in @not_merged")
lonnsutgifter.set_index(['komnr'], drop=True, append=False, inplace=True, verify_integrity=True)
lonnsutgifter=lonnsutgifter[["2020","2021"]].apply(pd.to_numeric, errors='coerce')
#lonnsutgifter.drop(['region', 'art','komname'],axis=1,inplace=True)
lonn_all.to_csv("lonn.csv")

In [10]:
konstra_group=import_data_excell(folder+"/KOSTRA-grupper_2020.xlsx",Type=str,Header=0)[0]
#konstra_group=konstra_group.replace({"komnr":dict_kode_int},regex=True)
konstra_group.set_index(['komnr'], drop=True, append=False, inplace=True, verify_integrity=True)
konstra_group.to_csv("kostra_group.csv")
print(konstra_group)

               komnavn kostragr
komnr                          
0301              Oslo       13
1101         Eigersund        7
1103         Stavanger       12
1106         Haugesund       10
1108           Sandnes       12
...                ...      ...
5440          Berlevåg       15
5441       Deatnu-Tana        6
5442   Unjárga-Nesseby       15
5443         Båtsfjord        6
5444      Sør-Varanger        8

[356 rows x 2 columns]


#check this dataset 

In [1]:
timer_i_uka=pd.read_excel(folder+"/SSB/Merged serie/12292 Gjennomsnittlig antall tildelte timer i uken - 2020-2021.xlsx",dtype=str,header=2,index_col=[0,1])#[0]
timar_i_uka_67plus=timer_i_uka.xs('Gjennomsnittlig antall tildelte timer i uken, hjemmetjenestebrukere 67 år og over (antall)').apply(pd.to_numeric, errors='coerce')
timar_i_uka_67plus[['komnr','komname']]=timar_i_uka_67plus.index.to_series().str.replace("K-", '',regex=False).str.split(" ", n = 1, expand = True)
timar_i_uka_67plus=timar_i_uka_67plus.replace({"komnr":dict_kode},regex=True).groupby(by="komnr", axis=0, level=None, as_index=True, sort=False).sum()

timar_i_uka_total=timer_i_uka.xs('Gjennomsnittlig antall tildelte timer i uken, helsetjenester i hjemmet (antall)').apply(pd.to_numeric, errors='coerce')
timar_i_uka_total[['komnr','komname']]=timar_i_uka_total.index.to_series().str.replace("K-", '',regex=False).str.split(" ", n = 1, expand = True)
timar_i_uka_total=timar_i_uka_total.replace({"komnr":dict_kode},regex=True).groupby(by="komnr", axis=0, level=None, as_index=True, sort=False).sum()
timar_i_uka_total=timar_i_uka_total.query("komnr in @kode.GRUPPEKODE and komnr !='Rest'")
timar_i_uka_total.to_csv("timar_i_uka.csv")
timar_i_uka_67plus=timar_i_uka_67plus.query("komnr in @kode.GRUPPEKODE and komnr !='Rest'")
timar_i_uka_67plus.to_csv("timar_i_uka_67plus.csv")

NameError: name 'pd' is not defined

In [ ]:
vakter=import_data_excell(folder+"/NSF/Vikarbruk alle kommuner 2012-2020.xlsx",Type=str,Header=0)[0]
vakter[['komnr','komname']]=vakter["Kommune"].str.replace("K-", '',regex=False).str.split(" ", n = 1, expand = True) 
vakter=vakter.replace({"komnr":dict_kode},regex=True)
vakter.drop(['Endring i utgifter  fra 2019-2020',
       'Prosentvis endring i utgifter_x000D_\n',
       ' Vikarutgift per innbygger (2020) ', 'Unnamed: 14', 'Unnamed: 15',
       'komname', "Kommune"],axis=1,inplace=True)
vakter.columns=[w.replace("Totale utgifter ", '') for w in vakter.columns]
vakter=vakter.query("komnr in @kode.GRUPPEKODE")
vakter.set_index(['komnr'], drop=True, append=False, inplace=True, verify_integrity=False)
vakter_all=vakter.apply(pd.to_numeric, errors='coerce')
vakter_all.to_csv("vakter.csv")
vakter=vakter_all#.query("komnr in @not_merged") 

In [ ]:
pai1=import_data_excell(folder+r"/PAI/Sykefravær per kommune.xlsx",Type=str)[0][["aar","komnr","Kjonn_tekst", "sykeprosent"]]
pai1["combo"]=pai1["Kjonn_tekst"]+pai1["aar"].astype(str)
pai1=pai1.replace({"komnr":dict_kode},regex=True)
#pai1=pai1.replace({"komnr":dict_kode_int},regex=True)
# pai_syke=pai1.query("Kjonn_tekst == 'Begge kjønn'").query("komnr in @kode.GRUPPEKODE").pivot(index=['komnr'],columns='aar',values='sykeprosent')
# pai_syke=pai_syke.apply(pd.to_numeric, errors='coerce')
# pai1.set_index(['komnr','aar',"Kjonn_tekst"], drop=True, append=False, inplace=True, verify_integrity=True)

pai2=import_data_excell(folder+r"/PAI/Stillingsstørrelse per kommune.xlsx", Type=str)[0]
pai2.Tid=[parser.parse(w).replace(tzinfo=None) for w in pai2.Tid]
pai2["aar"]=[w.year for w in pai2.Tid]
pai2["komname"]=pai2["kommune"].str.replace(" kommuner", '')
pai2["komname"]=pai2["komname"].str.replace(" kommune", '')
pai2=pai2.replace({"Komnr":dict_kode_int},regex=True)
pai2.drop(['Tid','kommune','komname'],axis=1,inplace=True)
pai_stilstor= pai2.pivot(index=['Komnr'],columns='aar',values='stilstor').apply(pd.to_numeric, errors='coerce').query("Komnr in @kode.GRUPPEKODE")
pai_stilstor_all= pai2.pivot(index=['Komnr'],columns='aar',values='stilstor').apply(pd.to_numeric, errors='coerce')

pai_heltid=pai2.pivot(index=['Komnr'],columns='aar',values='andel_heltidsansatte').query("Komnr in @kode.GRUPPEKODE").apply(pd.to_numeric, errors='coerce')
pai_heltid_all=pai2.pivot(index=['Komnr'],columns='aar',values='andel_heltidsansatte').apply(pd.to_numeric, errors='coerce')

pai2.set_index(['Komnr','aar'], drop=True, append=False, inplace=True, verify_integrity=True)

pai_heltid=pai_heltid.apply(pd.to_numeric, errors='coerce')
pai_stilstor=pai_stilstor.apply(pd.to_numeric, errors='coerce')
# pai_syke=pai_syke.apply(pd.to_numeric, errors='coerce')
pai_heltid.index.name='komnr'
pai_stilstor.index.name='komnr'
pai_heltid_all.to_csv("heltid.csv")
pai_stilstor_all.to_csv("stilstor.csv")

In [ ]:
ncr_report=import_data_excell(folder+r"/NRK Brennpunkts kartlegging av kommunal omsorg(1-176).xlsx",Header=0)[0]
ncr_all=ncr_report[['Kommunenummer:','Hvor mange tjenesteavvik hadde hjemmetjenesten totalt i 2016?',
'Hvor mange tjenesteavvik hadde hjemmetjenesten totalt i 2017?',
'Hvor mange tjenesteavvik hadde hjemmetjenesten totalt i 2018?',
'Hvor mange tjenesteavvik hadde hjemmetjenesten totalt i 2019?',
'Hvor mange tjenesteavvik hadde hjemmetjenesten totalt i 2020?',
'Hvor mange tjenesteavvik hadde hjemmetjenesten totalt i 2021?']].apply(pd.to_numeric, errors='coerce')
ncr_med=ncr_report[['Kommunenummer:','Hvor mange tjenesteavvik hadde hjemmetjenesten som gjelder medisin-, legemiddel- eller medikamenthåndtering 2016?'
,'Hvor mange tjenesteavvik hadde hjemmetjenesten som gjelder medisin-, legemiddel- eller medikamenthåndtering i 2017?',
'Hvor mange tjenesteavvik hadde hjemmetjenesten som gjelder medisin-, legemiddel- eller medikamenthåndtering i 2018?',
'Hvor mange tjenesteavvik hadde hjemmetjenesten som gjelder medisin-, legemiddel- eller medikamenthåndtering i 2019?',
'Hvor mange tjenesteavvik hadde hjemmetjenesten som gjelder medisin-, legemiddel- eller medikamenthåndtering i 2020?',
'Hvor mange tjenesteavvik hadde hjemmetjenesten som gjelder medisin-, legemiddel- eller medikamenthåndtering i 2021?']].apply(pd.to_numeric, errors='coerce')

name_columns=['komnr','2016','2017','2018','2019','2020','2021']
ncr_med.to_csv("med_NCR.csv")
ncr_all.to_csv("all_ncr.csv")

In [52]:
kpr_old=import_data_excell(folder+r"/Helsedir/Leveranse 2/21_38387 KPR kommunale tjenestetyper 2017-2020/21_38387 KPR kommunale tjenestetyper 2017-2020.xlsx",Type=str,Header=2)[0]


KeyboardInterrupt: 

In [ ]:
kpr["Funksjon"]=kpr["Funksjon"].str.replace("FUNKNIVAA", '',regex=False)
kpr_pivot=kpr.pivot(index='Funksjon', columns='aar', values='MEAN').apply(pd.to_numeric, errors='coerce')#'K75PROS'
selection=["1","3","4","5","7","8","10","11","14","15","17","16","13"]
kpr_pivot=kpr_pivot.loc[selection]
kpr_pivot.csv("KPR_data.csv")

In [47]:
kpr['antall brukere']=kpr['antall brukere'].astype(float)
kpr['Gjennomsnittlig funksjonsverdi']=kpr['Gjennomsnittlig funksjonsverdi'].astype(float) 
KPR_stats=kpr.groupby(['kommunenummer',"aldersgruppe","tjenestetype - tekst", "aar","funksjonstype - tekst"]).apply(lambda x: np.average(x["Gjennomsnittlig funksjonsverdi"], weights=x['antall brukere'])).reset_index()
KPR_stats_all_ages=kpr.groupby(['kommunenummer',"tjenestetype - tekst", "aar","funksjonstype - tekst"]).apply(lambda x: np.average(x["Gjennomsnittlig funksjonsverdi"], weights=x['antall brukere'])).reset_index()


In [51]:
KPR_stats.rename(columns={0:'mean_value'}, inplace=True)
KPR_stats_all_ages.rename(columns={0:'mean_value'}, inplace=True)
KPR_stats.to_csv("kpr_age_devided.csv")
KPR_stats_all_ages.to_csv("kpr.csv")


In [10]:
Users_by_sickness=import_data_excell(folder+r"/SSB/Merged serie/12292 Hjemmeboende brukere - bistandsbehov 2015-2021.xlsx",Header=2,col_index=[0,1])[0].apply(pd.to_numeric, errors='coerce').reset_index()

Users_by_sickness[['komnr','komname']]=Users_by_sickness["level_1"].str.split(" ", n = 1, expand = True) 
Users_by_sickness.drop(["level_1", "komname"],inplace=True, axis=1)
Users_by_sickness.columns=Users_by_sickness.columns.astype(str)
Users_by_sickness=Users_by_sickness.replace({"komnr":dict_kode},regex=True)
users_by_sickness=Users_by_sickness.groupby(by=['komnr',"level_0"],axis=0,as_index=False).sum(min_count=1)
users_by_sickness.rename(columns={'level_0':'variable'}, inplace=True)
users_by_sickness=users_by_sickness.query("komnr in @kode.GRUPPEKODE")
users_by_sickness.drop("Rest",inplace=True)
users_by_sickness.to_csv("users_by_sickness.csv")

In [13]:
users_very_sick=users_by_sickness[users_by_sickness.variable.str.contains("i alt")]
users_very_sick.set_index(['komnr'], drop=True, append=False, inplace=True, verify_integrity=True)
total_users=users_by_sickness[~users_by_sickness.variable.str.contains("i alt")]
total_users2=total_users.groupby(by='komnr',axis=0,as_index=True).sum(min_count=1)
users_very_sick=users_very_sick.divide(total_users2).drop("variable", axis=1)
users_very_sick.drop("Rest",inplace=True)
users_very_sick.to_csv("users_very_sick.csv")